In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
API_KEY = 'sRcVocmkQXDjHVFqtgBH'

In [ ]:
import subprocess
import json

def fetchPredictions(image, api_key, version=2, confidence=40):
  '''
  run inference using yolov8 model trained on matrices
  returns a list of prediction objects extracted from a json
  parameters:
    image: filename of the image (uploaded to google colab)
    api_key: roboflow api key
    version: the trained model to be used. latest one trained on the most data right now is 2
    confidence: the lower threshold for a prediction to be accepted. default value is 40
                but may often need to be lowered to the 20-30 range to capture all numbers
  '''
  command = 'base64 ' + image + ' | curl -d @- "https://detect.roboflow.com/matrix-classifier/'
  command = command + str(version) + '?api_key=' + API_KEY + '&confidence=' + str(confidence) + '"'

  result = subprocess.getoutput(command)
  # Parse the JSON response
  json_start = result.find('{"inference_id":')
  if json_start != -1:
      # Extract the JSON string
      json_str = result[json_start:]

      try:
          # Parse the JSON response
          response_json = json.loads(json_str)
          print("Successfully retrieved JSON")  # Now you can access the JSON object
      except json.JSONDecodeError:
          print("Failed to decode JSON.")
  else:
      print("JSON part not found in the output.")

  return response_json['predictions']


In [ ]:
print("\nDo you have one or 2 matrices?")
numOfMatrices = input("Enter '1' or '2': ").strip().lower()
if numOfMatrices == '1':
  predictions = []
  #THIS PART CALLS API FROM DEPLOYED AI MODEL ON ROBOFLOW
  predictions.append(fetchPredictions('matrix1.jpg', API_KEY))
  #THIS PART CALLS API FROM DEPLOYED AI MODEL ON ROBOFLOW
else:
  predictions = []
  #THIS PART CALLS API FROM DEPLOYED AI MODEL ON ROBOFLOW
  predictions.append(fetchPredictions('matrix1.jpg', API_KEY))
  predictions.append(fetchPredictions('matrix2.jpg', API_KEY))
  #THIS PART CALLS API FROM DEPLOYED AI MODEL ON ROBOFLOW




Do you have one or 2 matrices?
Enter '1' or '2': 1
Successfully retrieved JSON


In [ ]:
import numpy as np
from sympy import *

answers = []
print(len(predictions))

for pred in predictions:

  min_x = float('inf')
  min_y = float('inf')
  max_x = float('-inf')
  max_y = float('-inf')


  for item in pred:
      if item['x'] < min_x:
          min_x = item['x']
      if item['y'] < min_y:
          min_y = item['y']
      if item['x'] > max_x:
          max_x = item['x']
      if item['y'] > max_y:
          max_y = item['y']


  grid_width = (max_x - min_x) / 3
  grid_height = (max_y - min_y) / 3
  matrix = [["" for _ in range(3)] for _ in range(3)]
  filled_cells = [[False for _ in range(3)] for _ in range(3)]
  def find_closest_empty_cell(row, col):
      for r in range(3):
          for c in range(3):
              if not filled_cells[r][c]:
                  return r, c
      return None, None
  for item in pred:
      col = int((item['x'] - min_x) // grid_width)
      row = int((item['y'] - min_y) // grid_height)
      col = min(col, 2)
      row = min(row, 2)
      if filled_cells[row][col]:
          row, col = find_closest_empty_cell(row, col)
      matrix[row][col] = item['class']
      filled_cells[row][col] = True
  for r in range(3):
      for c in range(3):
          if not filled_cells[r][c]:
              row, col = find_closest_empty_cell(r, c)
              matrix[r][c] = matrix[row][col]
              filled_cells[r][c] = True


  print("Matrix:")
  for row in matrix:
      print(row)


  answers.append(matrix)




1
Matrix:
['5', '1', '1']
['4', '3', '2']
['1', '3', '6']


In [ ]:
print("\nDo you want to add, subtract, or multiply the two matrices?")
operation = input("Enter 'add', 'subtract', or 'multiply', or 'determinant': ").strip().lower()

if numOfMatrices == '2':
  matrix1 = np.array(answers[0], dtype=int)
  matrix2 = np.array(answers[1], dtype=int)
  if operation == "add":
    result = np.add(matrix1, matrix2)
  elif operation == "subtract":
    result = np.subtract(matrix1, matrix2)
  elif operation == "multiply":
    result = np.multiply(matrix1, matrix2)
else:
  matrix1 = np.array(answers[0], dtype=int)
  if operation == "determinant":
    result = np.linalg.det(matrix1)



print("Result:")
print(result)


Do you want to add, subtract, or multiply the two matrices?
Enter 'add', 'subtract', or 'multiply', or 'determinant': determinant
Result:
46.99999999999999
